<a href="https://colab.research.google.com/github/Gulov009/TrabalhoTC---Ot-vio/blob/main/T_C_projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import e
import pandas as pd
from collections import deque

"""
##############################################AUTÔMATOS##############################################
"""

#==========================================AFD=======================================================

"""
definição do Autômato Finito Determinístico (AFD), a partir da solicitação ao usuário
que especifique cada um dos componentes
(Alfabeto, Estados, Estado Inicial, Conjunto de Estados Finais, e a Função de Transição);
"""

# cabecalho: M = (Σ, Q, δ, q0, F)

def AutomatoFinitoDeterministico():
  print("--- Configuração do Autômato Finito Determinístico (AFD) ---")

  # 1. Definição dos Estados (Q)
  estados = input("Informe os estados separados por espaço (ex: q0 q1 q2): ").split()

  # 2. Definição do Alfabeto (Σ)
  alfabeto = input("Informe os símbolos do alfabeto separados por espaço (ex: a b): ").split()

  # 3. Função de Transição (δ)
  transicoes = {}
  print("\nAgora, defina as transições para cada estado e símbolo:")
  for estado in estados:
    transicoes[estado] = {}
    for simbolo in alfabeto:
      while True:
        destino = input(f"  δ({estado}, {simbolo}) -> vai para qual estado? (Deixe vazio se não for para nenhum)")
        if destino == "":
          transicoes[estado][simbolo] = '-'
          break
        elif destino in estados:
          transicoes[estado][simbolo] = destino
          break
        else:
          print(f"  Erro: O estado '{destino}' não pertence ao conjunto de estados informado.")

  # 4. Estado Inicial (q0)
  while True:
    estado_inicial = input("\nInforme o estado inicial: ")
    if estado_inicial in estados:
      break
    print("Erro: O estado inicial deve estar entre os estados informados.")

  # 5. Estados Finais (F)
  estados_finais = input("Informe os Estados Finais separados por espaço: ").split()
  estados_finais = [e for e in estados_finais if e in estados]

  exibir_tabela_transicao(estados, alfabeto, transicoes, estado_inicial, estados_finais)

  return {
        'tipo': 'AFD',
        'estados': estados,
        'alfabeto': alfabeto,
        'transicoes': transicoes,
        'estado_inicial': estado_inicial,
        'estados_aceitacao': estados_finais
    }

#==========================================AFD=======================================================



#========================================AFND========================================================
def AutomatoFinitoNaoDeterministico():
  print("--- Configuração do Autômato Finito Nao Determinístico (AFND) ---")

  # 1. Definição dos Estados (Q)
  estados = input("Informe os estados separados por espaço (ex: q0 q1 q2): ").split()

  # 2. Definição do Alfabeto (Σ)
  alfabeto = input("Informe os símbolos do alfabeto separados por espaço (ex: a b): ").split()

  # Adicionamos o epsilon ao alfabeto interno para transições vazia
  alfabeto_extendido = alfabeto + ['&']

  transicoes = {}
  print("\nDefina as transições (se houver mais de um destino, separe por espaço. Deixe vazio para nenhum):")
  for estado in estados:
    transicoes[estado] = {}
    for simbolo in alfabeto_extendido:
      destino = input(f"  δ({estado}, '{simbolo}') -> vai para quais estados? ").split()
      # Validar se os estados existem
      destinos_validos = [d for d in destino if d in estados]
      transicoes[estado][simbolo] = destinos_validos

  estado_inicial = input("\nEstado inicial: ")
  estados_finais = input("Estados finais (separados por espaço): ").split()
  estados_finais = [e for e in estados_finais if e in estados]
  exibir_tabela_afn(transicoes, estados, alfabeto, estado_inicial, estados_finais)

  return {
      'tipo': 'AFND',
      'estados': estados,
      'alfabeto': alfabeto,
      'transicoes': transicoes,
      'estado_inicial': estado_inicial,
      'estados_aceitacao': estados_finais
  }

#========================================AFND========================================================



#======================================ValidarPalavra================================================
"""
validação de uma palavra w, informada pelo usuário - armazenada em uma fita de entrada -, e validada pelo AFD definido anteriormente.
A entrada é a palavra w e a saída é composta pelo processo de reconhecimento ou não - indicação do passo a passo de leitura do símbolo da fita
a partir do estado atual da máquina e consequente transição de estado -, e ao final, a mensagem se a palavra é válida ou inválida.
"""
def ValidarPalavra(config):
  # Estas linhas foram corrigidas para 4 espaços de indentação
    print("\n" + "="*30)
    palavra = input("Informe a palavra para testar (ou 'sair' para encerrar): ")

    if palavra.lower() == 'sair': return

    if config['tipo'] == 'AFD':
        estado_atual = config['estado_inicial']
        erro_alfabeto = False
        caminho = [estado_atual]

        for simbolo in palavra:
            if simbolo not in config['alfabeto']:
                print(f"Erro: O símbolo '{simbolo}' não pertence ao alfabeto {config['alfabeto']}.")
                erro_alfabeto = True
                break

            # Transição de estado
            estado_atual = config['transicoes'][estado_atual].get(simbolo, '-')
            caminho.append(estado_atual)

            if estado_atual == '-':
              break

        if not erro_alfabeto:
            print(f"Caminho percorrido: {' -> '.join(caminho)}")
            if estado_atual in config['estados_aceitacao']:
                print(f"Resultado: A palavra '{palavra}' foi ACEITA! (Estado final: {estado_atual})")
            else:
                print(f"Resultado: A palavra '{palavra}' foi REJEITADA. (Estado final: {estado_atual})")

    elif config['tipo'] == 'AFND':
      estados_atuais = {config['estado_inicial']}

      caminho_log = [f"Início: {estados_atuais}"]
      for simbolo in palavra:
        proximos_estados = set()

        for estado in estados_atuais:
          destinos = config['transicoes'].get(estado, {}).get(simbolo, [])
          proximos_estados.update(destinos)

        estados_atuais = proximos_estados
        caminho_log.append(f"Lendo '{simbolo}': {estados_atuais}")

        if not estados_atuais:
          break

      print("\n--- Rastreamento dos Estados Ativos ---")
      for passo in caminho_log:
          print(passo)
      print("---------------------------------------")

      if not estados_atuais.isdisjoint(config['estados_aceitacao']):
        print(f"Resultado: A palavra '{palavra}' foi ACEITA!")
        print(f"Estados finais possíveis: {estados_atuais}")
      else:
        print(f"Resultado: A palavra '{palavra}' foi REJEITADA.")
        print(f"Estados finais alcançados: {estados_atuais}")

#======================================ValidarPalavra================================================


#======================================TabelaTransiçãoAFD===============================================
def exibir_tabela_transicao(estados, alfabeto, transicoes, estado_inicial, estados_aceitacao):
    """
    Gera e exibe a tabela de transição de um AFD usando Pandas.
    """
    dados = []
    indices = []

    for estado in estados:
        # Define os marcadores visuais (-> para inicial, * para final)
        marcador = ""
        if estado == estado_inicial:
            marcador += "→ "
        if estado in estados_aceitacao:
            marcador += "* "

        # Cria o rótulo da linha (ex: "→ * q0")
        indices.append(f"{marcador}{estado}")

        # Preenche as transições para cada símbolo do alfabeto
        linha = []
        for simbolo in alfabeto:
            # Busca o destino; se não houver transição (o que não deve ocorrer em AFD), põe '-'
            destino = transicoes.get(estado, {}).get(simbolo, '-')
            linha.append(destino)
        dados.append(linha)

    # Cria o DataFrame
    df = pd.DataFrame(dados, columns=alfabeto, index=indices)
    df.index.name = "Estado (δ)"

    print("\n--- Tabela de Transição (Função δ) ---")
    # Exibe o dataframe formatado
    print(df)
    return df

#======================================TabelaTransiçãoAFD============================================



#=======================================TabelaTransiçãoAFND==========================================

def exibir_tabela_afn(transicoes, estados, alfabeto, inicial, aceitacao):
    dados = []
    indices = []

    for estado in estados:
        # Formatação visual dos índices
        marcador = ""
        if estado == inicial: marcador = "→ "
        if estado in aceitacao: marcador += "* "
        indices.append(f"{marcador}{estado}")

        linha = []
        for simbolo in alfabeto:
            destinos = transicoes[estado][simbolo]
            # Formata a lista ['q0', 'q1'] como "{q0, q1}" ou "-"
            if destinos:
                cell = "{" + ", ".join(destinos) + "}"
            else:
                cell = "-"
            linha.append(cell)
        dados.append(linha)

    df = pd.DataFrame(dados, columns=alfabeto, index=indices)
    df.index.name = "Estado"
    print("\n--- Tabela de Transições (AFN) ---")
    print(df)

#=======================================TabelaTransiçãoAFND==========================================

#==========================================Epsilon_fecho=============================================
#GEMINI QUE FEZ!

def calcular_epsilon_fecho(estados, transicoes):
    """
    Recebe uma lista de estados e o dicionário de transições do autômato.
    Retorna uma tupla ordenada contendo o Epsilon-Fecho desses estados
    (todos os estados alcançáveis através de transições '&', incluindo eles mesmos).
    """
    # Se a lista estiver vazia, retorna uma tupla vazia
    if not estados:
        return ()

    closure = set(estados)
    fila_eps = deque(estados)

    while fila_eps:
        est = fila_eps.popleft()
        # Busca os destinos partindo do estado atual consumindo apenas '&'
        destinos_epsilon = transicoes.get(est, {}).get('&', [])

        for d in destinos_epsilon:
            if d not in closure:
                closure.add(d)
                fila_eps.append(d)

    return tuple(sorted(closure))

#==========================================Epsilon_fecho=============================================

#=======================================ConversãoAFNDParaAFD===========================================
#USEI O GEMINI PARA ENTENDER ALGUMAS COISAS Ex: Como juntar o conjunto de estados possíveis (q0, a -> q0, q1) em um único estado (q0, a ->{q0q1})

def conversao_afnd_para_afd(afnd):
  estado_inicial_afd = calcular_epsilon_fecho([afnd['estado_inicial']], afnd['transicoes'])

  fila = deque([estado_inicial_afd])
  transicoes_afd = {}
  estados_visitados = {estado_inicial_afd}

  print(f"Iniciando conversão partindo de: {formatar_estado(estado_inicial_afd)}")

  while fila:
    estado_atual_tuple = fila.popleft()
    transicoes_afd[estado_atual_tuple] = {}

    for simbolo in afnd['alfabeto']:
      novos_destinos = set()

      for sub_estado in estado_atual_tuple:
        destinos = afnd['transicoes'].get(sub_estado, {}).get(simbolo, [])
        novos_destinos.update(destinos)

      destino_afd = calcular_epsilon_fecho(list(novos_destinos), afnd['transicoes'])

      transicoes_afd[estado_atual_tuple][simbolo] = destino_afd

      if destino_afd not in estados_visitados and destino_afd:
        estados_visitados.add(destino_afd)
        fila.append(destino_afd)

  #Função para renomear (mapeamento) os novos conjuntos de estados obtidos
  mapa_nomes = {}
  contador = 0

  estados_descobertos = [estado_inicial_afd] + [e for e in transicoes_afd.keys() if e != estado_inicial_afd]

  print("\n======= Legenda de Renomeação dos Estados =======")
  for estado_composto in estados_descobertos:
    if estado_composto not in mapa_nomes:
      novo_nome = f"Q{contador}"
      mapa_nomes[estado_composto] = novo_nome
      print(f"{novo_nome} representa o conjunto {formatar_estado(estado_composto)}")
      contador += 1

  transicoes_renomeadas = {}
  novos_estados = list(mapa_nomes.values())
  novo_inicial = mapa_nomes[estado_inicial_afd]
  novos_finais = []

  for estado_composto, caminhos in transicoes_afd.items():
    nome_origem = mapa_nomes[estado_composto]
    transicoes_renomeadas[nome_origem] = {}

    for simbolo, destino_composto in caminhos.items():
      if destino_composto:
        nome_destino = mapa_nomes[destino_composto]
        transicoes_renomeadas[nome_origem][simbolo] = nome_destino
      else: #Aqui eu bati muito a cabeça porque minha função de transição tava sempre preenchendo a coluna do último símbolo do alfabeto como um '-'
            #até que eu descobri (perguntei pro gemini) que tava faltando uma identação criando um estrutura for-else ao invés de uma if-else
        transicoes_renomeadas[nome_origem][simbolo] = '-'


    eh_final = any(estado in afnd['estados_aceitacao'] for estado in estado_composto)
    if eh_final and nome_origem not in novos_finais:
      novos_finais.append(nome_origem)

  exibir_tabela_transicao(novos_estados, afnd['alfabeto'], transicoes_renomeadas, novo_inicial, novos_finais)

  return {
        'tipo': 'AFD',
        'estados': novos_estados,
        'alfabeto': afnd['alfabeto'],
        'transicoes': transicoes_renomeadas,
        'estado_inicial': novo_inicial,
        'estados_aceitacao': novos_finais
    }

def formatar_estado(estado_tuple):
  if not estado_tuple: return '∅'
  return "{" + ','.join(estado_tuple) + "}"


#=======================================ConversãoAFNDtoAFD===========================================

#=========================================MinimizaçãoAFD=============================================

def minimizacao_afd(afd):
  alfabeto = afd['alfabeto']
  transicoes = afd['transicoes']
  estado_inicial = afd['estado_inicial']
  estados_aceitacao = set(afd['estados_aceitacao'])

# Tornando a tabela do AFD completa (Substituindo todos os "buracos" da tabela por um estado sintético qualquer)
  todos_estados = list(set(afd['estados'] + list(transicoes.keys())))


  estado_morto = "X" # X -> estado morto qualquer
  morto_necessario = False

  for est in todos_estados:
    for simbolo in alfabeto:
      if simbolo not in transicoes.get(est, {}) or transicoes[est][simbolo] == '-':
        morto_necessario = True
        if est not in transicoes: transicoes[est] = {}
        transicoes[est][simbolo] = estado_morto

  if morto_necessario:
    todos_estados.append(estado_morto)
    transicoes[estado_morto] = {simbolo: estado_morto for simbolo in alfabeto}

# Agora que preenchemos os espaços vazios da tabela, antes de irmos para a parte de procurar os estados equivalentes
#é preciso também removermos logo os estados que são inatingíveis. Pra fazer isso usei busca em largura para visitar todos
#os vizinhos que os estados tem, começando do estado inicial e indo até o último estado possível a ser alcançado.
  atingiveis = {estado_inicial}
  fila = deque([estado_inicial])

  while fila:
    atual = fila.popleft()
    for simbolo in alfabeto:
      destino = transicoes[atual][simbolo]
      if destino not in atingiveis:
        atingiveis.add(destino)
        fila.append(destino)

  estados_validos = [e for e in todos_estados if e in atingiveis]

# Particionamento - Inicialmente a primeira partição contará tanto com os estados finais quanto com os não finais
  grupo_finais = tuple(sorted([e for e in estados_validos if e in estados_aceitacao]))
  grupo_nao_finais = tuple(sorted([e for e in estados_validos if e not in estados_aceitacao]))
  particao = []

  if grupo_finais: particao.append(grupo_finais)
  if grupo_nao_finais: particao.append(grupo_nao_finais)

  while True:
    nova_particao = []
    for grupo in particao:
      if len(grupo) <= 1:
        nova_particao.append(grupo)
        continue

      #
      subgrupos = {}
      for est in grupo:
        assinatura = []
        for simb in alfabeto:
          destino = transicoes[est][simb]

          for i, g in enumerate(particao):
            if destino in g:
              assinatura.append(i)
              break

        assinatura = tuple(assinatura)
        if assinatura not in subgrupos:
          subgrupos[assinatura] = []
        subgrupos[assinatura].append(est)
      for sg in subgrupos.values():
        nova_particao.append(tuple(sorted(sg)))

    if len(nova_particao) == len(particao):
      break
    particao = nova_particao

# Reconstrução do AFD minimizado
  mapa_representante = {}
  novos_estados = []

  for i, grupo in enumerate(particao):
    nome_novo = f"M{i}"

    estados_antigos = "{" + ", ".join(grupo) + "}"
    print(f"{nome_novo} representa o conjunto: {estados_antigos}")

    for est in grupo:
      mapa_representante[est] = nome_novo
    novos_estados.append(nome_novo)

  novas_transicoes = {}
  for grupo in particao:
    representante = grupo[0]
    nome_novo = mapa_representante[representante]
    novas_transicoes[nome_novo] = {}
    for simbolo in alfabeto:
      destino_antigo = transicoes[representante][simbolo]
      novas_transicoes[nome_novo][simbolo] = mapa_representante[destino_antigo]

  novo_inicial = mapa_representante[estado_inicial]
  novos_finais = sorted(list(set(mapa_representante[e] for e in estados_aceitacao if e in atingiveis)))

  print("\n======= Minimização Concluída =======")
  exibir_tabela_transicao(novos_estados, alfabeto, novas_transicoes, novo_inicial, novos_finais)

  return{
      'tipo': 'AFD',
      'estados': novos_estados,
      'alfabeto': alfabeto,
      'transicoes': novas_transicoes,
      'estado_inicial': novo_inicial,
      'estados_aceitacao': novos_finais
  }


#=========================================MinimizaçãoAFD=============================================

"""
##############################################AUTÔMATOS##############################################
"""

"""
##############################################GRAMÁTICA##############################################
"""


"""
##############################################GRAMÁTICA##############################################
"""

#========================================Menu========================================================

def menu():
  maquina_atual = None  # Variável que guardará as definições do automato (deterministico ou nao)
  gramatica_atual = None # Variável que ira armazenar a gramática

  while True:
    print("\n===== MENU DE AUTÔMATOS ====")
    print("0 - Sair")
    print("1 - Definir AFD")
    print("2 - Definir AFND")
    print("\n===== MENU DE GRAMÁTICAS ====")
    print("3 - Definir Gramática Regular")
    print("4 - Definir Gramática Livre de Contexto")
    print("\n======= MENU DE AÇÕES =======")
    print("----------AUTOMATOS----------")
    print("5 - Validar Palavra (AFD ou AFND)")
    print("6 - Converter AFND para AFD")
    print("7 - Realizar Minimização (Apenas se a máquina atual for um AFD)")
    print("8 - Exibir tabela de transição da máquina atual")
    print("----------GRAMÁTICA----------")
    escolha = input("Escolha uma opção: ")

    if escolha == "1":
      maquina_atual = AutomatoFinitoDeterministico() # Captura o retorno do AFD
      print("\n[SUCESSO] AFD configurado e salvo na memória!")
    elif escolha == "2":
      maquina_atual = AutomatoFinitoNaoDeterministico()
      print("\n[SUCESSO] AFND configurado e salvo na memória!") # Passa o AFD definido como argumento
    elif escolha == "5":
      if maquina_atual is None:
        print("\n[ERRO] Nenhum autômato configurado! Defina um AFD (1) ou AFND (2) primeiro.")
      else:
        ValidarPalavra(maquina_atual)
    elif escolha == "6":
      if maquina_atual is None:
        print("\n[ERRO] Nenhum autômato configurado! Defina um AFND (2) primeiro.")
      elif maquina_atual['tipo'] != 'AFND':
        print("\n[AVISO] A máquina atual já é um AFD. Não é necessário converter.")
      else:
        maquina_atual = conversao_afnd_para_afd(maquina_atual)
        print("\n[SUCESSO]Conversão de AFND para AFD concluída com sucesso!")
    elif escolha == "7":
      if maquina_atual is None:
          print("\n[ERRO] Nenhum autômato configurado! Defina ou converta para um AFD primeiro.")
      elif maquina_atual['tipo'] != 'AFD':
          print("\n[ERRO] A máquina atual é um AFND. Converta para AFD (Opção 4) antes de minimizar.")
      else:
          print("\nIniciando Minimização...")
          maquina_atual = minimizacao_afd(maquina_atual)
    elif escolha == "8":
      if maquina_atual is None:
          print("\n[ERRO] Nenhum autômato configurado para exibir.")
      else:
          # Chama a função de exibição correta dependendo do tipo da máquina
        if maquina_atual['tipo'] == 'AFD':
          exibir_tabela_transicao(
            maquina_atual['estados'],
            maquina_atual['alfabeto'],
            maquina_atual['transicoes'],
            maquina_atual['estado_inicial'],
            maquina_atual['estados_aceitacao']
          )
        else:
          exibir_tabela_afn(
            maquina_atual['transicoes'],
            maquina_atual['estados'],
            maquina_atual['alfabeto'],
            maquina_atual['estado_inicial'],
            maquina_atual['estados_aceitacao']
          )
    elif escolha == "0":
      print("Saindo do programa...")
      break
    else:
      print("Opção inválida! Digite um número de 1 a 6.")
if __name__ == "__main__":
  menu()


===== MENU DE AUTÔMATOS ====
0 - Sair
1 - Definir AFD
2 - Definir AFND

===== MENU DE GRAMÁTICAS ====
3 - Definir Gramática Regular
4 - Definir Gramática Livre de Contexto

======= MENU DE AÇÕES =======
----------AUTOMATOS----------
5 - Validar Palavra (AFD ou AFND)
6 - Converter AFND para AFD
7 - Realizar Minimização (Apenas se a máquina atual for um AFD)
8 - Exibir tabela de transição da máquina atual
----------GRAMÁTICA----------
Escolha uma opção: 1
--- Configuração do Autômato Finito Determinístico (AFD) ---
Informe os estados separados por espaço (ex: q0 q1 q2): q0 q1
Informe os símbolos do alfabeto separados por espaço (ex: a b): a b

Agora, defina as transições para cada estado e símbolo:
  δ(q0, a) -> vai para qual estado? (Deixe vazio se não for para nenhum)q1
  δ(q0, b) -> vai para qual estado? (Deixe vazio se não for para nenhum)q0
  δ(q1, a) -> vai para qual estado? (Deixe vazio se não for para nenhum)q0
  δ(q1, b) -> vai para qual estado? (Deixe vazio se não for para nen

KeyboardInterrupt: Interrupted by user